In [1]:
import pandas as pd
import datetime
import calendar

In [2]:
# Read the file in as a pandas dataframe
rawdata_df = pd.read_csv('White goods fires from 2009.csv')

In [3]:
# Drop unwanted fields
droppedFields_df = rawdata_df.copy()
droppedFields_df.drop(['IncGeo_BoroughCode','IncGeo_WardCode','IgnitionSourcePower'],axis=1, inplace=True)

In [4]:
# Best practice to copy the df
Caps_clean = droppedFields_df.copy()

# List of column headers
Col_Head = list(Caps_clean.columns)

# Convert values to uppercase, handling null values
for col in Col_Head:    
    
    # Check if column dtype is integer
    if Caps_clean[col].dtype == 'int64':
        continue  # Skip columns with integer dtype
    
    else:    
        Caps_clean[col] = Caps_clean[col].apply(lambda x: x.lower() if pd.notnull(x) else x)

In [5]:
# Best practice to copy the df
NullWords_clean = Caps_clean.copy()

# List of words which mean null
NullWordsList = ['unknown', 'fit to supply', 'not known', 'unable to find', 'unable to read burnt', 'undetermined', 'unknown-no evidence', 'not determinable', 'badly damaged', 'unable to identify ', 'unidentified', 'not determined', 'no make or model number', 'no manufacturers details available', 'no marks or identification', 'unable to identify', 'not identified', 'believed philips', 'not yet determined', 'imported appliance from uae', 'unable to positively identify manufacturer', 'unable to read label', 'faulty programer ', 'unable to determine make', 'not known - dishwasher', 'too badly damaged', 'no distinguishing marks on fridge', 'unkmown', 'tbc', 'unknown severly fire damaged', 'uknown', 'unbranded', 'no markings', 'fridge completely destroyed by fire', 'hda', 'unknown (destroyed)', 'possibly hotpoint or indisit', 'unknown ', 'no known', 'see further information', 'very old appliance, manufacturer not known by owner and unmarked', 'unknown due to damage', 'not confirmed', 'unkown', ' unknown', 'unknown - no identifying marks', 'other', 'commercial fridge, no make  visible', 'could not find name', 'damaged beyond recognition', 'possibly bought from argos', 'unidentifiable', 'not known due to the age and extent of burning', 'unknown burnt out', 'manufacturer unclear upon examination of appliance', '.', 'unknown total loss', 'unbranded industrial unit', 'comercial freezer', 'dometic', 'n/k ', 'make ', '0', 'no idenity markings on freezer', 'n/a', 'no manufacturers name available', 'semiautomatic washing machine', 'pending confimation by owner', 'pending further information ', 'awaiting analysis', 'item not known due to fire damage', 'unconfirmed hotpoint/whirpool', 'un identfiable', 'unconfirmed possibley beko', 'not located due to fire damage. owner unsure of manufacturer.  ', 'not ', 'was unable to identify', 'not known ', 'unknown make', 'essential', 'pro action ', 'unk nown', 'n/k', 'unknown no markings', 'hotpoint or indesit - dxf', 'pending', 'unknown - old and worn off label', 'currys', 'not visible', 'commercial fridge', 'unknown due to fire damage', 'storer display counter refridgerator', 'illedgble label','completely burnt out', 'unkown ', 'essentials', 'unknown all markings removed', 'old spin drier', 'polar refrigeration', 'no details available', 'not known, identification destroyed by fire', 'subcold', 'not available', 'awaiting bv examination', 'not shown ', 'baumatic tumble dryer', 'old spin dryer - brand not known', 'innex', 'pending amdea report', 'adexa', 'easy cold ce', 'no manufacturer found ', 'custom made fridge', 'tcym750c6p', 'no make visible']

# Replace words with null in the entire DataFrame
NullWords_clean.replace(NullWordsList, pd.NA, inplace=True)

In [6]:
# Best practice to copy the df
Concat_df = NullWords_clean.copy()

# Create a new list
ApplianceManufacturerList = []

# Append to the lists
for value1, value2 in zip(Concat_df['ApplianceManufacturer'], Concat_df['ApplianceManufacturerOther']):
    if pd.isnull(value1) and pd.isnull(value2):
        ApplianceManufacturerList.append(pd.NA) # Append null if both fields are null
    elif pd.isnull(value1):
        ApplianceManufacturerList.append(value2) # Append ApplianceManufacturer if null in other field
    elif pd.isnull(value2):
        ApplianceManufacturerList.append(value1) # Append ApplianceManufacturerOther if null in other field
    else:
        newValue = value1 + value2
        ApplianceManufacturerList.append(newValue) # concatenate if both fields are not null


# Replace ApplianceManufacturer field with new values
Concat_df['ApplianceManufacturer'] = ApplianceManufacturerList

# Drop ApplianceManufacturerOther field
Concat_df.drop('ApplianceManufacturerOther',axis=1, inplace=True)

In [7]:
# Best practice to copy the df
Clean_df = Concat_df.copy()

# Capitalising function
def capitalise(Column):
    ColumnList = [] # create new list
    for value in Clean_df[Column]:
       if pd.isnull(value): # append with null if null
           ColumnList.append(value)
       else:
           CapValue = value.capitalize() # append capitalised value
           ColumnList.append(CapValue)
    Clean_df[Column] = ColumnList

# Capitalise fields with propoer noun values
capitalise('Month')
capitalise('IncGeo_BoroughName')
capitalise('IncGeo_WardName')
capitalise('ApplianceManufacturer')

# Function to convert worded month to its numerical representation
def month_to_number(month_name):
    return list(calendar.month_name).index(month_name.capitalize())

# Apply the function to the 'Month' column
Clean_df['Month_number'] = Clean_df['Month'].apply(month_to_number)

# Combine 'Year' and 'Month' columns and create a new 'Date' column
Clean_df['Date'] = Clean_df['Year'].astype(str) + '-' + Clean_df['Month_number'].astype(str).str.zfill(2)

# Convert 'Date' column to datetime format
Clean_df['Date'] = pd.to_datetime(Clean_df['Date'], format='%Y-%m')

Clean_df.drop(['Month_number'],axis=1, inplace=True)

# Write DataFrame to CSV file
Clean_df.to_csv('clean_firedata.csv', index=False)